In [46]:
import os
import random
import json
import time
from io import BytesIO
import cv2
import base64
import logging
import uuid
import traitlets
import numpy as np
from gym_donkeycar.core.sim_client import SDClient


###########################################

class SimpleClient(SDClient, traitlets.HasTraits):
    image = traitlets.Any()

    def __init__(self, address, racer, vehicle, poll_socket_sleep_time=0.01, *args, **kwargs):
        traitlets.HasTraits.__init__(self, *args, **kwargs)
        host, port = address
        SDClient.__init__(self, host, port, poll_socket_sleep_time=poll_socket_sleep_time)
        self.last_image = None
        self.car_loaded = False
        self.racer = racer
        self.vehicle = vehicle

    def is_loaded(self):
        return self.car_loaded

    def on_msg_recv(self, json_packet):
        if json_packet['msg_type'] == "need_car_config":
            self.send_config(self.racer, self.vehicle)

        if json_packet['msg_type'] == "car_loaded":

            self.car_loaded = True
        
        if json_packet['msg_type'] == "telemetry":
            imgString = json_packet["image"]
            self.image = cv2.imdecode(np.frombuffer(base64.b64decode(imgString), dtype=np.uint8), cv2.IMREAD_COLOR)

            #don't have to, but to clean up the print, delete the image string.
            del json_packet["image"]

            if "imageb" in json_packet:
                imgString = json_packet["imageb"]
                np_img = cv2.imdecode(np.frombuffer(base64.b64decode(imgString)))
                #image = Image.open(BytesIO(base64.b64decode(imgString)))
                #image.save("camera_b.png")
                #np_img = np.asarray(image)
                print("imgb:", np_img.shape)

                #don't have to, but to clean up the print, delete the image string.
                del json_packet["imageb"]

            if "lidar" in json_packet:
                lidar = json_packet["lidar"]
                if lidar is not None:
                    print("lidar:", len(lidar), "pts")

                #don't have to, but to clean up the print, delete the lidar string.
                del json_packet["lidar"]


    def send_config(self, racer, vehicle):
        '''
        send three config messages to setup car, racer, and camera
        '''
        racer_name = racer
        car_name = vehicle
        bio = "I race robots."
        country = "France"
        guid = str(uuid.uuid4())

        # Racer info
        msg = {'msg_type': 'racer_info',
            'racer_name': racer_name,
            'car_name' : car_name,
            'bio' : bio,
            'country' : country,
            'guid' : guid }
        self.send_now(json.dumps(msg))
        time.sleep(0.2)
        
        # Car config
        # body_style = "donkey" | "bare" | "car01" choice of string
        # body_rgb  = (128, 128, 128) tuple of ints
        # car_name = "string less than 64 char"

        msg = '{ "msg_type" : "car_config", "body_style" : "car01", "body_r" : "255", "body_g" : "0", "body_b" : "255", "car_name" : "%s", "font_size" : "100" }' % (car_name)
        self.send_now(msg)

        #this sleep gives the car time to spawn. Once it's spawned, it's ready for the camera config.
        time.sleep(0.2)

        # Camera config
        # set any field to Zero to get the default camera setting.
        # this will position the camera right above the car, with max fisheye and wide fov
        # this also changes the img output to 255x255x1 ( actually 255x255x3 just all three channels have same value)
        # the offset_x moves camera left/right
        # the offset_y moves camera up/down
        # the offset_z moves camera forward/back
        # with fish_eye_x/y == 0.0 then you get no distortion
        # img_enc can be one of JPG|PNG|TGA        
        msg = '{ "msg_type" : "cam_config", "fov" : "150", "fish_eye_x" : "1.0", "fish_eye_y" : "1.0", "img_w" : "255", "img_h" : "255", "img_d" : "1", "img_enc" : "JPG", "offset_x" : "0.0", "offset_y" : "3.0", "offset_z" : "0.0", "rot_x" : "90.0" }'
        self.send_now(msg)
        time.sleep(0.2)

        # Camera config B, for the second camera
        # set any field to Zero to get the default camera setting.
        # this will position the camera right above the car, with max fisheye and wide fov
        # this also changes the img output to 255x255x1 ( actually 255x255x3 just all three channels have same value)
        # the offset_x moves camera left/right
        # the offset_y moves camera up/down
        # the offset_z moves camera forward/back
        # with fish_eye_x/y == 0.0 then you get no distortion
        # img_enc can be one of JPG|PNG|TGA
        msg = '{ "msg_type" : "cam_config_b", "fov" : "150", "fish_eye_x" : "1.0", "fish_eye_y" : "1.0", "img_w" : "255", "img_h" : "255", "img_d" : "1", "img_enc" : "JPG", "offset_x" : "3.0", "offset_y" : "3.0", "offset_z" : "0.0", "rot_x" : "90.0" }'
        self.send_now(msg)
        time.sleep(0.2)

        # Lidar config
        # the offset_x moves camera left/right
        # the offset_y moves camera up/down
        # the offset_z moves camera forward/back
        # degPerSweepInc : as the ray sweeps around, how many degrees does it advance per sample (int)
        # degAngDown : what is the starting angle for the initial sweep compared to the forward vector
        # degAngDelta : what angle change between sweeps
        # numSweepsLevels : how many complete 360 sweeps (int)
        # maxRange : what it max distance we will register a hit
        # noise : what is the scalar on the perlin noise applied to point position
        # Here's some sample settings that similate a more sophisticated lidar:
        # msg = '{ "msg_type" : "lidar_config", "degPerSweepInc" : "2", "degAngDown" : "25", "degAngDelta" : "-1.0", "numSweepsLevels" : "25", "maxRange" : "50.0", "noise" : "0.2", "offset_x" : "0.0", "offset_y" : "1.0", "offset_z" : "1.0", "rot_x" : "0.0" }'
        # And here's some sample settings that similate a simple RpLidar A2 one level horizontal scan.
        msg = '{ "msg_type" : "lidar_config", "degPerSweepInc" : "2", "degAngDown" : "0", "degAngDelta" : "-1.0", "numSweepsLevels" : "1", "maxRange" : "50.0", "noise" : "0.4", "offset_x" : "0.0", "offset_y" : "0.5", "offset_z" : "0.5", "rot_x" : "0.0" }'
        self.send_now(msg)
        time.sleep(0.2)


    def send_controls(self, steering, throttle):
        msg = { "msg_type" : "control",
                "steering" : steering.__str__(),
                "throttle" : throttle.__str__(),
                "brake" : "0.0" }
        self.send(json.dumps(msg))

        #this sleep lets the SDClient thread poll our message and send it out.
        time.sleep(self.poll_socket_sleep_sec)

    def update(self):
        # just random steering now
        st = random.random() * 2.0 - 1.0
        th = 0.3
        self.send_controls(st, th)



In [47]:
def test_clients():
    logging.basicConfig(level=logging.DEBUG)

    # test params
    host = "127.0.0.1" # "trainmydonkey.com" for virtual racing server
    port = 9091
    num_clients = 1
    clients = []
    time_to_drive = 30.0


    # Start Clients
    for _ in range(0, num_clients):
        c = SimpleClient((host, port), "Pedro", "Alpine")
        clients.append(c)

    time.sleep(2)

    # Load Scene message. Only one client needs to send the load scene.
    msg = '{ "msg_type" : "load_scene", "scene_name" : "mountain_track" }'
    clients[0].send_now(msg)


    # Send random driving controls
    start = time.time()
    do_drive = True
    while time.time() - start < time_to_drive and do_drive:
        if not c.is_loaded():
            time.sleep(0.3)
            continue
        for c in clients:
            c.update()
            if c.aborted:
                print("Client socket problem, stopping driving.")
                do_drive = False

    time.sleep(3.0)

    # Exit Scene - optionally..
    msg = '{ "msg_type" : "exit_scene" }'
    for c in clients:
        c.send_now(msg)

    # Close down clients
    print("waiting for msg loop to stop")
    for c in clients:
        c.stop()

    print("clients to stopped")

In [48]:
test_clients()

INFO:gym_donkeycar.core.client:connecting to 127.0.0.1:9091 
DEBUG:gym_donkeycar.core.client:send_now:{ "msg_type" : "load_scene", "scene_name" : "mountain_track" }
DEBUG:gym_donkeycar.core.client:send_now:{"msg_type": "racer_info", "racer_name": "Pedro", "car_name": "Alpine", "bio": "I race robots.", "country": "France", "guid": "ee0df122-10ae-451e-8ae5-a6d2578d0f0c"}
DEBUG:gym_donkeycar.core.client:send_now:{ "msg_type" : "car_config", "body_style" : "car01", "body_r" : "255", "body_g" : "0", "body_b" : "255", "car_name" : "Alpine", "font_size" : "100" }
DEBUG:gym_donkeycar.core.client:send_now:{ "msg_type" : "cam_config", "fov" : "150", "fish_eye_x" : "1.0", "fish_eye_y" : "1.0", "img_w" : "255", "img_h" : "255", "img_d" : "1", "img_enc" : "JPG", "offset_x" : "0.0", "offset_y" : "3.0", "offset_z" : "0.0", "rot_x" : "90.0" }
DEBUG:gym_donkeycar.core.client:send_now:{ "msg_type" : "cam_config_b", "fov" : "150", "fish_eye_x" : "1.0", "fish_eye_y" : "1.0", "img_w" : "255", "img_h" : "25

DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.19446002794712158", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.5081927633780865", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.7174731916439188", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.5292850516995029", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.6777637405079888", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.9600439992659382", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.5299773310451201", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "s

DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.7304365900476275", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.873538539179989", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.4647433683787363", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.3705260634303629", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.1089530706223174", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.7190757745236664", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.965497314549838", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steer

DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.4379626538431771", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.8793963231513044", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.8541763983103703", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.4645156208939858", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.8384739620392063", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.5623707784349312", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.6975942026485615", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "ste

DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.28718983604122417", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.24528676950670025", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.29530355224643046", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.725064600229633", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.9577010001259694", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.9311578581200364", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.9006232935457814", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control

...

DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.33081023629535", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.14356477206251794", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.28481090165000045", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.564186908324481", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.281878720678161", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.08298973088794193", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.2654653800346791", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "st

DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.5668421427544843", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.6867980062476682", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.8960577507009557", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.5633898524279461", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.9873964567077533", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.461752774837245", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.20430080992916544", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", 

DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.1871165097774692", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.44323809938397973", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.18436673102579237", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.8868088231560729", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.21370367430659631", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.9736904332791474", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.8044978682061623", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control

DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.2598350831310141", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.31708066102060206", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.24271358583354008", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.29888387222399215", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.1605108008643792", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.2180548390271122", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.5333472592644539", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control"

waiting for msg loop to stop
clients to stopped


In [33]:
SDClient??

In [32]:
json_msg = '{"msg_type":"telemetry","steering_angle":0,"throttle":0,"speed":0.8855376,"image":"/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjL/wAARCAD/AP8DASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwDZMgij3d6oMxuJcdcmlv58ttSpLJQF3nrQBo24WKIKO1ZeoXBlmESH61Ld3ghQgHrVfToDNJ5r+tAGpZQeVD05xUgUscVKD8uBR/q42fv2+tAFW4w2FDbUBwx/nUXmFph5YYR4wM0y4VvJCJyxPOamTJu7aBBkkgUAbloy2ekXEY4fZuY+56VyjvkbevP5mun8Qulpa+VGRvl5J9BXKE8k+g/KgBjHLH64FKOox+H+NMGMn0HH/wBanZxnP44/lQA9T0I/D/GnA46dunufWmHkH0PB+npTuS3HBP6CgB6kY9QD+ZpsmPLOfq3+H86EPAI99tNl+77DH4mgCLJL+5P5D/P86UEDB7L096ZjnryT19KcrYw2OB0HrQBatiRcIM/MW5PpWrqi77YkfcQj8T/n+dY0ZKzKAfmLDJravcNYsq/6tF/P/OKAMXJA3dSelPBIUKD8x6mo1PG5uvQClUlQP7xoAlX/AJ5r0HWpQdw2L90darglflXqetSo2BsXr3NAE4cg4UZAFRnlzG3Mb9D6UucAIvXvTwE2hD19fSgBscZ27G+8nQ1Jnac/nTnIVcgZIqrJNtxIoyp4I9KAHSy7MOOUPX2qHPzGNvuOPlNO2YJB5ikH5UscWAYm5A6GgAijLKYZOo+6amWPzBtcfMvQ0qr0BPzCn78/d6+lAGXFGZ5Sx6CrLjygSOKtw24ijxiqOoSBRsXqaAM1t11cY7A10NnGIowvbFZtjb4+Y960g+zFAFuL5nxUN5cJ5hiVuUGTj1p4kEMDTYyQOB6msqMO1w8jYUOc+pNAEq7kgJ5L9cVr6HbZP2hxmRRhR71mW7CON2k4LnCgda6K2C2enNJ02Lk49fSgDD125Et+yg7gnf1P+FYztgDHfn/69STOZHJPOTk/4VBIMnJ7/rQAL29f880oPAI57gepoAz2znrTtpY9MZ6+woAdnCjuB+ppx4Vuen3jTchTyyjnA56CkaeAAjzVwP1NAEq5wfU8fQf5zSSkEewOBTFuIQmC5yTkkCkluIEKMZOvQelAEfOdvryTQDwG9BwP8/hUZmhyF85M9+etOEickSIcD5RmgCVSc+5IzXQkCWzxn92E5Pqa5tc7QFOSepFdJbjzLNAQQij86AMFTxubt0FKCV5PLHpTWwsr56AkAUoOPmPLHoKAHg7RgcsetSIdgwOXPWoQdp9WJp6N5eOfnNAEwPljA5c1IpEYyTlj0qNf3ec8uaUDZ8z8k9KAFlMiKJPfke1NVNshOMxv29KlxvXe/C+lKjKcgDAFAAkW1Sh6dqQtg7f4h0ppkLEx9G7VAztMhxxKn60AOMpcEr99eopCzOoli+90YU0LvImXhhwwqdF2vuQYB6igC7cOIoyTWJGhu7okjjNT6ndtI/krjmrunW3lRAnqetAEiQiNAKPL3ECp5F54pygJGXP4UAU7mVIysZOQvb3qsQxuN+cDt7Ut0o3Bndc9QBVc3IiBKrn2P86AL1rEbm+jTnA9Ow/xrX1+cWmmJG5CGTjBPIGOar+E4nmklun4QcKO1ZPjG+8/UxCDlYxj8aAMt7uMZ2hmJ4z0xVSW/kZ/lCrjjgVEx2qTVdDk5NAFgzysRukY/jTzLI/3nY/jUAPNPHJ9qAJM9+9KANlIMDJNJyVxnrQBOv3KgnPAz61Mv3RVW4PIHvQBVfJkzQCaD1P1poOBQBIpwMgnj0rtNIndtMjw56Y61xSrx1xmut0Jv+JdjOcGgDHu7yaK/lQkEBsjIqSG/jZszDae2OlQasu3U39xVPOKAN6OaJgxSRWb0zzUi/KckfMTXPKec1YiuZo2G2Q49D0oA3R8nJ5YjpUgIX5n59BWbHqBDZeNWPcjg1bjuYHOWYg+jUAWQDINzHC+lBJbhRgetNQFyXLAqP7vSjcXG1eF9aAGSoGZcN869/WpRGGIcDDd6aWVT8oyaHlaJQxwR3FADvlQk+tMaYRgHGVqKRyrhgMxtSKjI5B5jbkUAY2j3xvpzJMnAPGK6+KZNnyngetcppluLaBQB2rR+0gDac0AdAnzmqepTMJUhRsKBluOpqK1ujFG0hbcAOAaozyrMzPcSbAeoHWgBk0zMSqYOOpqGTcdsYHzMcUlvIksh8g/u17+taOjQm81iPK5wcn2oA6+whGmaICRjYhYn3rzS8mae7lmJyWYmvRvFV19l0R0BwZPlH0rzFzz9aAEkOUqEHFPlbAAFQg9aAJQckVKp54qunWpUJwaAFacLcLEFyWGSfSpmP3ahGM5xzipC3IzxQBN2+lVJfvA1OWqvcHkUAVyTnHvSL6UjHmlDcUASryRXT6Cc20i+9csh4BrovDzkmUe1AFXXFC3ykdStZ1a3iFf3sTdDWPzigB+KUZzTQeOaXODQBZU5HNTKehzVZCeOKnU80ASrI0bhkLD6VaW8cAK43L9MGqff0FPPIFAGms0TLiJwG/2uKcVCD5/mJ5FZBBHXmnRzSxN8jsPbtQBpxqACpGFPQHtT9yLhT+tZ5v2zmRMj1U4qQ3EFygAk2N/tcUAKINq4pvlZPSrjx88U+CP5ue1AFWZlgRY889cVTuI+hIUk+vaqiXbzTz3MrJseRvK3N0UcD+VSq5l+Ygf8B5oAnRjDa4zy1dL4OgBMs2Oema5jy3YDg+mTXe+HbYWmlr6tyaAMLxxdZeK3B6ckVxLNyBW54nuDcatJzkDgVhAfNzQBFKfnxmkWh/vGlA4oAF4qVelMHSnnhaAHZ7CnHAIpg6ign9560AT5qpcHJGDVnNQTAAA5oArcUgxj8aRjzSgcUASjpW74dP+kOp/u1gr0rb8PnF4R6rQBY8QrmKNsdGrCrodfXNqD6GueoActL34pFHFGcGgCePnGKnHoarxnmrA/SgCQindBTOtPB4oAQnjmmkgMO9OIytRscNQA5wDnJqLGTxUr4bnpUGcN1oA3vN+fDGp55RHaMw4LfKKzUkkLDKg1Dfzyy6pDaoSY44DI6DjJJwP5GgCqbGAPvZAx7Z6flVuEhZFGQFHYDpUDkoo+UKfTOcU8MrICG57jFAGgksFxcRxKpPzY5Ndy5FpprFcDalcHoEIn1iP/ZOa7HXp/I0iXBxkYoA80u5zLcySMcksaqs5ALU9+SST1NRsuUNAFcMSSalGQKjAwcCpMDPNACjIHNSfw/jTMcjFSEfKKAFx0oOPM45pwHTNN2jzMUAS81WuAcis/UNTkSfy4srtPOagXVJZGAYDA5NAF4/rSg8dKbHIsihl6HvTlHWgCUf6sGtXQWAv1+lZQOFxWlohxqCUAbOvJusGPvmuXB9K6rWhusHx2rlU+6DQA9DTjUY608dKAJVqwvAziqyk9uasKeBxQA/04qRTx0qPPSpF6UAIx/D2pjmnuB2FMIw3tQApOUBxURALZqXgDOKhPXOQRQBuwQhpAeaqSwBNVuZy2S4VcY6BR/8AXNa0KKDuHYViNPFchpjKPKZjgjPPOKACWVQdqqSfbpUbsCmcmopJbeIgRPknqCamlA8lRt2sR60AbnhCDfcyTk9BWh4xnEemqmeWNJ4PgxaSNkdap+OBxCuegoA4rkjimSkKmKf0HSo5hwKAGLzzTxTF4p4OKAHgU/sOKZ1xUgHSgAzuIA7UudrFjzgUdGAHNNkYANnGMdTQBzV5KJbp3AwCelV/LkcnavT3xSTOUmfI6Ems+XU5S2VIA9OtAHQ2l20CJFMuOcAjmtJcEE5rj7XU5PPQPhgSB0rp0nUXBhDZbGaALYPz1o6Mx/tBMVnJgn3rS0Yf8TFKAOi1ZSdPk+lcinArr9V/48JPpXIJ0oAcBzThkGkwc04YoAcuQanHaoFGT0qwOwoAf0OalT5sioxj61KmM9aAEIHemEYqU4qNgBjuaAE6dqicjPSpjkdTxUT/AHs0AbVvcKIpGBzhSeawNHPmaZBvJClM8H15rdWFjbTheWMbAfXFc1aGSCzhhihZ2VQrmMg4I6jPSgC68amUYVcA9SOasXTcqoAHFV4lkK5ZGB9zzUrxSdSrZ+lAHbeE0Caazd2asbxu3+koO22trwpGY9ObcDkt3rC8bn/S0A/u0AcgcAimynpSseRUcpwaAGgnNKGyRmkHUUo+9QBMDzUnbk1COvNSZ5FAEq4J/CqsrbsqeR6VYBPt0qjdMwRynLY4FAGJf/Z/P2pndj5/Ss8aPE78yOoJ/Kp42NzdyMeD1Oe1aEJhnX906uR1IOaAMyDSIUlDb3YqcgHoa6A/Y1tgwDfa8gE9sVUZoYHAllRCRkZNIsomkLjGAcAjoaANOD5hk1taKg/tCI471iWoyBiug0cYvouO9AG3qw/0GQYPSuRRflFdlqw/0GQjHSuQUfL0oAbg04AilHXml7cUAKn3qnXOR3qFfpVhex5oAeOacmNxFIBxT1HXNADdvNI3YZxTyB3pp4FADW+7n9aiJIPXNTsBj2xUIXnigDooFGevBrmLJDG13GduFupBz9a24rhVkHz5FVpo7X7bKiBN7kylepye9AFYZPbp6VbN1F5QR93PoarSQMrh1I/WmygFenPpQB3fhvadOBUnGe9c941IN4o9q1/CMoaxdM8hulZfjdMTRv6igDjmXnNRygcU88n2qGbORigAUc08LmmICcVJznpQAoXmpMcikU88in4+YUAKgzn6VVlTJIAq6oxmmLGWzgcUAcfPbzCSddo+Y8AGr/h/QItQiumnlaFoyAMMFz19at3OmyC985EyCeSD7UhiPYZNADdb0WLTrKAwv5rvIQW3BjjHTiq0CpG6268PtDEVbEbYJIxVmLT4jKtxyZSoHtigCW2iwBW9oqE38YPaqEUGFFbWhRZvhxQBo6yNtg/vXJAEAV1+v/LZYzjJrltvAoAhAzzS457VKEzSMvNACIvNTquBwaYikGpwBigAA46U4DjFGKeAQOlADCpIqMj5sVKTz0pgB3bqAGv9KhAPWppM+lM7d6AHrCiyF9g3NgEipp4YotlwsSLKwKs4UAt9TUmxc8ipZowbVTgYV+hoAy2e4dscbPUHFPIO0gj8aklvI0PlypFt7Arj9RUVtKWZgIgUPcNnFAG74SvY4bp4HJBbpxxUvjee3CxqbiISL1QuM/lWNpJ8rWIyBwTXT6x4J0XxPP8AaL2aeC6ACgowAI7cEUAeeblYfKwP0NJKoKgkc11qeDLTwAX1+O5e7hhU7oWUcg+9SQ/EzwjeELdacsZPXfCG/pQBxSLzg1IqjNaHiLSNYudQe/8AD+nmbTZvnj8uPIx9B0qLw1pl9qGqG01m2ksI8f6xkKjP40AV1HzCpccrXdf8K+tnGYNULf8AAAf5GvO9c1GLRNWksJEaQxnG8cZ/CgC+g5OB2pYlJfJp/hz/AIqSVorL/WIuSrVtDwxrEc7D7DKR/eAGDQBkm3DqfSqMlmEfA6davTX9pa3DwS3MaSKcMrHkGp4LdtSUvaoZQo5Kc0AZaWuTllBHoaspAAoAAq21rLbsUnQoSOA3FKmzaoyM0AMMWEXArY8PR5vCcdBVFInlHyIxx6DNbvh6B1ncupU44yMUAReJwotUHctXLlc11XipQBCCRn61z6wO4+VGb6DNAFQCgjNWWtZkBZonCjqSpAqtLNDCheSRUA6k0APUDIqcCp7TT7y7jEkFtLJGejqpwfxqefTLq0t3nuYTFEoyXkIAFAFMLzUmMVnHW9NU4+1xn6ZP8q6yx8M6hqNnFdQLGIZV3KXbGR9KAMAj1puAK1/EGjyeHdO+2X0ylC20LGMkmuQj8RC5lEdtYTyZOBxj+WaANRgM8U0KK6628O6KltHPqWspC7Llk3qu32yar6no9pPYj/hFsanOG+ZhKCqj65AoAyDGQamCFoJVx/DkfhSvgKCefqalhIODxj0oAwvL81WPGAee4qKyt4orsiMhSw6BuKtHbBdTRLuVwefQioAFWbdtKtnmgCeNzb3inuGBr0diLjTf95M15rNkTBs5FejaM4m0iE/7OKAPM9QluRJPbNPK0JJBjZyVI+lYraXYyctbLkHsSK6PxFCYdVlGOpzWSB60AdJo/jeXSrSKzSzQwRjCjdzTvFPiqLxD4elsYreS3uGZWV8jHHX3rkyMMakFAGRbw6/aSK0F9IFBB4kr1ey0zwvrOnW76msT3jIN7FiCTj2rgkOVqdenHWgDptetbHwHp66p4bjHnO2xgWLjFZOn/FvV5LmKGeziO9guQMVW5MeG5X0NKsUIlVhGmev3RmgDoL34T/2xcNqKagI2uP3hUjpnmpbGOH4UKo1BvtovCdnl8bdvrn61Xt9Wv7dQI7qUAdAWJqprVzLrghXUCJfKzsOOmev8qAOwsviPousyiyis3WaYFELKDya5m7+F2vT3Us0cke12LAbsf1rG0+1j07UIrqBAskbZXPNdtbePNWUES+XJzx8gXA/CgB+h3dh4Dszp+vMftMnzAIu8Y+tag8UaJrsU1jpSZvJI2WMmMDBIx1rj9emXxFeC6uokWQLtG0mr3g3TILLUWuol/eIOM8igDCn+GHim4kLPOGySRukP9TW9pnibR/A9gNE1USS30JzJ5aBhzyOc+lb2t+KdR0+dUjKYI7rivPtUtINWv5r24hUzSnLEfTFAHWHxBZ+O7G60rSLdoJiufMkUAAfhXOx/Bi9Zw02px5zk/Kf/AK9QaVLLoRkbT38hpBhmUAkirr67qzsGfUJz9X4/KgCJPiSvhuH+x7PTPtH2Y7PMMmA2O+MVs6XqUnxG0yeDU4lsrVDzsOCfxNc40KOSzAFjyeKkVFVcDhfQdKANw+APBmngs1z5soGVXzskn6Vyk+veM8mC1ultbZCViRdvC9vXtWgqjk9aQoeuQaANXw1q6Q2En/CSyTajclsorDeF+meB2rQvfFtmttJBY6QkZdCqyHAK5HXAH9a5cgE9fzqKTGSKAMc6BYs7O5nkcnJLOMH8hXS6Nq13odj9l04pBHnOdoYn881nbecg1Ls4BIoAu8npzTEl2viora53xKy9CKdJIoOSDmgCHVikMqyswQOvLCs+Ft+T5yOvY961LqMXljtyQVPX2rLt9NigfIY8dRQBO+5gCe1dp4Ru/Ns3gPWM8fSuPWI4OB8tbfhKYxamydA4xQBF4zt/LvVlH8Q5rlRw3PevQ/Glt5mnrMByprzwn1oAik4ahTgc0snWkUdcdKAJUx65qwmOlV4+MirEfSgCXBI+hqdV+YHv2qFcZJ7+9T4y6+poAtoCTyKhc/vVqdQ3br71AcmUZHNADovvHIOSatW4G0c+tQRAYyfXirMPzouD1oAeg27s9PUiul8LpzMwrGSPAAxk11PhmMCOU9+OKAMHxPh9RVcHIFYTLiug8QfPqz57VkSRg9KAKJGegBpCvPODVhk9qbsOaAGbTxT8etSKhz0xSlO/egCFQQDnNNPpirJTgd6jYelAFfaOSeDUbrlqsZ4yahPJz2oAj2DIHNPAwOcUqjdyCRT1XI9aAMPRLp3tFQnOK0nQtySBXM6Y7Wd6Ym4BNdUmGANAEtqoyVJJDDBqsw+Zw2cqcZI/rVmMEOCMUX6xxt5r7QHHX3oAit3ZHAD/ACN61pQSva6jDJ5RC5GW6/rWSoI6Y9fara3u0rkAj0oA7jWIPtekSjGcrkV5TKm1ip7HFet6fIt1pUbAj5lxXmer2/kajNGP71AGZJ90GmrxgVMVJQiol7UAPQfN0qeMfNxUOMEVMnX0oAlT361ZBB2HHXiq6HqDzVlMKFIH1oAtqBjH5VGy4deRmnqVwCOeabIDvXHrQAQ9QvTOeavQJ8g3c47mqMefMA7Voo3yr2JxQBdhTJGM8etdf4dhxayMeOa5aFcYIwa7LQxt0wkjqSaAOQ1Q+dqUxzxuxWe6DOAa0br95dzMDwWNVJI+aAKjqcZxUW09hmrTRkDnOKYEPYUAR7OORSlT6ZqcJjAxSbfmNAEDAVCw7VbK57fjUDLlj7UAQSbQvp+FQDrnIqzIvao9oJ4oAaFwOmaeoyPSn7c9KcqcjP5AUAcfqtq0EizqOh5rb0ucTwKcAnFO1C1FxbEdeOKy9DnFrdtbS568UAdAeDxU8kaXFoVdQwHUHuKa0fcDipYB82D0IxQBkzxpER5TFcdB1okjygIxzzV+SPeCsiZ2kjIqqIzGG4Yoex5xQB1vhC6MtlJAx5jORXPeLbX7PqxkAwJOateFrjyNYEZOBIMVpeNrPfZpcAcocGgDgu5qDHzEe9TkHdUJyJDkUASKvORUijBpinK9qnQA49aAHjip0zsHTg8YqED5unUVMOIh9evpQBbTDKPeo5uGXI79akjAVAM596jnBJXoMUAPRT19DWhCP9WO1UY8EMM1oW2GdQeMCgDRi++vGa7iy/c6HuPGEJri7VdzjjvXaX2IfD7AnH7vH50AcWRnLepqN0zzjFWxGAowDUbR+xxQBRZMk89KFRgc1MwBzjkU5EGO5P06UAQlOOaZs9qnZcdBTXTjnr6UAVZPYiq+Bye9Wm5Iz/Kon2rxnmgCocZPHPrSryOTzTipzTwB3WgBgDAirVuYo3BlOBjJxUaru9APaql/E9phpTgvyAewoASMCWMEdCK5zV7drO8S6jHAPNauk3JdPJY8ireoWYurV0x2oAksbsXdqjrjGOlWFwr4rmtAuGgmks5DjB4ropDkg+lAD7hPmDqeH4I9xVOeRQ4Q8Me+KuRt5sTr1IGR9az5GdpAHXemeCo6UAEbvb3UcvQowORXfanCuo6C5HO6PeDXBOC0ZBzkdM13fhmRbrQ1jbkplSPagDy9o2VyO4qtKv7wE962tXtfsuqTwf3WOPpWbJFuTnqKAI1AxmpUyOtRqOMZ5qZRxQBMB8wPNSnAjIPrTF6D2qZU3A89qAJ4j8ntTplAAYc5PIogIMY46UTEZUe/agBsQPmVqQL+8GOcis63GG68mtO2X94OOgoA19Nj3XEYA5JrqPEbKmmKmcbnAA9awNCj8y+iHbdmtfxMdz2keeNxagDFKj8KY6jHXipSCeCeKCMLwKAKezrgH6Uuw47iptmSMDk0rqAvJI+lAFbAzxz7mmSKMcZz3NWdgznFQyj649KAKbKc8DNQSKPSrbLxxVVgXPJJ9hQBEq5PTpTtuRTthU8frTggLA9aAHQvDbOHmAKrziuT8T6tJfXZUHBbnA7DtW3rNxHBGeeQMtXEtIZZWlc8sc/SgDYYNY3YYcDNdJAwltw4OQRWdqlsJEJHWq+kXTBjbsenSgDL1aJ7HUFuk+6TzXR2sq3Nujr/ABCoNXtBPYtxyKy/D92QrwMTlDxQBuxkwzZzxVnFuiMWjb5j95DyKozyHbkVLC5mgx68YoAcsYdiiyrJxwcYP4jtW54OuTFezWrHhhkA+tcnNObd0kmJKZ2iVOGX2I71t2MrWmrwTqeCRn3FAEvjez8rU47gKMSryfcVypGQR7V6N42tRcaQk4+9EwP4GvPWwqE+lAFVBgnPephyMd6hRlbkdDU6jnmgCRfu5Pb0qwBuBPTio0X5T3qxHyo6UASW4yg6Zp1yg+XjnNLb8IR05p9wNyhscg0ARxLkAn1rUs87nP8As4GazIssh9QRWpCMbsfQfnQB0fheMteqc9FzirPiCUvq8UI/gj3fnS+FVBlkbuq1BqXz6/cnOCqqtAFfGMCggVKF96RhxwcUARBDknnApNpJ+7x6mrAX5R6mmupLYPH9KAK7KAOarSEHt0q9KMKMdKqOpYk9AO1AFOUnbwD+NVyOeeD6ValHPP4AVGE7jAoAj4C471LFsWN5JOAgyR6+1AXLYA/Gte70yGw0B766JKou/aO57UAeX69ctLceV3J3MPSskDnpUk8zXFxJM33nYn6UwB8gLjJoA//Z","hit":"none","time":0.4263283,"accel_x":-0.0009992212,"accel_y":0.995648,"accel_z":-0.07163021,"gyro_x":-0.001824873,"gyro_y":-4.36138E-06,"gyro_z":8.347623E-06,"gyro_w":0.9999985,"pos_x":51.31858,"pos_y":5.238001,"pos_z":50.19022,"vel_x":-0.0009885429,"vel_y":0.8829759,"vel_z":-0.0673003,"cte":2.881378,"activeNode":0,"totalNodes":251,"disqualified":0,"bestLapTimeMS":1000000,"currentLapTimeMS":0,"numLapsCompleted":0,"on_road":1,"progress_on_shortest_path":0.005619049}'

In [35]:
import json
j = json.loads(json_msg)

In [39]:
image = cv2.imdecode(np.frombuffer(base64.b64decode(j['image'])), cv2.IMREAD_COLOR)

In [50]:
import ipywidgets
from IPython.display import display

image_widget = ipywidgets.Image(format='jpeg')

In [51]:
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [52]:
image_widget.value = bgr8_to_jpeg(image)

display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [53]:
def update_image(change):
    image = change['new']
    image_widget.value = bgr8_to_jpeg(image)
 
logging.basicConfig(level=logging.DEBUG)

# test params
host = "127.0.0.1" # "trainmydonkey.com" for virtual racing server
port = 9091
num_clients = 1
clients = []
time_to_drive = 30.0


# Start Clients
for _ in range(0, num_clients):
    c = SimpleClient((host, port), "Pedro", "Alpine")
    c.observe(update_image, names='image')
    clients.append(c)

time.sleep(2)

# Load Scene message. Only one client needs to send the load scene.
msg = '{ "msg_type" : "load_scene", "scene_name" : "mountain_track" }'
clients[0].send_now(msg)


# Send random driving controls
start = time.time()
do_drive = True
while time.time() - start < time_to_drive and do_drive:
    if not c.is_loaded():
        time.sleep(0.3)
        continue
    for c in clients:
        c.update()
        if c.aborted:
            print("Client socket problem, stopping driving.")
            do_drive = False

time.sleep(3.0)

# Exit Scene - optionally..
msg = '{ "msg_type" : "exit_scene" }'
for c in clients:
    c.send_now(msg)

# Close down clients
print("waiting for msg loop to stop")
for c in clients:
    c.stop()

print("clients to stopped")

INFO:gym_donkeycar.core.client:connecting to 127.0.0.1:9091 
DEBUG:gym_donkeycar.core.client:send_now:{ "msg_type" : "load_scene", "scene_name" : "mountain_track" }
DEBUG:gym_donkeycar.core.client:send_now:{"msg_type": "racer_info", "racer_name": "Pedro", "car_name": "Alpine", "bio": "I race robots.", "country": "France", "guid": "803c275a-1fc6-472d-8673-ca59d1a60dec"}
DEBUG:gym_donkeycar.core.client:send_now:{ "msg_type" : "car_config", "body_style" : "car01", "body_r" : "255", "body_g" : "0", "body_b" : "255", "car_name" : "Alpine", "font_size" : "100" }
DEBUG:gym_donkeycar.core.client:send_now:{ "msg_type" : "cam_config", "fov" : "150", "fish_eye_x" : "1.0", "fish_eye_y" : "1.0", "img_w" : "255", "img_h" : "255", "img_d" : "1", "img_enc" : "JPG", "offset_x" : "0.0", "offset_y" : "3.0", "offset_z" : "0.0", "rot_x" : "90.0" }
DEBUG:gym_donkeycar.core.client:send_now:{ "msg_type" : "cam_config_b", "fov" : "150", "fish_eye_x" : "1.0", "fish_eye_y" : "1.0", "img_w" : "255", "img_h" : "25

DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.26855758683867226", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.9596882751692293", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.013686680172404486", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.10521353533178601", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.4041282592621056", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.30651599502470717", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.13951710376062665", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "contr

DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.7382045010866742", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.7888444063313766", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.6101425099362285", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.2321261374837269", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.7746729468461424", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.4814964230073844", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.560373662988509", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "s

DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.2711755281072119", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.08108070253121724", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.7090122129516274", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.7201811058801293", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.8012156656062959", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.7712781316070569", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.963590113283612", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "

DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.6475417366576361", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.5845869133647585", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.6267155596337248", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.7613667460017564", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.5348535342344423", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.7482575604340365", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.07319412115310042", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "

...

DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.35631966041366625", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.10094942147954566", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.3589773614725198", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.36424040668004487", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.4240931535205228", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.47294051217612143", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.7579489065872647", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control"

DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.5929466754686215", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.8926730552991606", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.7902407550241297", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.7784419839230743", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.6987779285565243", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.9609850084869553", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.38358433264080927", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control"

DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.09192489895463352", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.3313496351838363", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.26080145209471284", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.6090939447985646", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.5835352260750581", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.5851214397382065", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.07082307907656316", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control",

DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.19949083478710183", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.8643972653939274", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.7568141223925358", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.9778267858669063", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "0.32701014117498306", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.14027358160406234", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control", "steering": "-0.42133618234091674", "throttle": "0.3", "brake": "0.0"}
DEBUG:gym_donkeycar.core.client:sending {"msg_type": "control

waiting for msg loop to stop
clients to stopped
